<a href="https://colab.research.google.com/github/GeorgeSakketos/Data_Mining_2025/blob/main/Data_Mining_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mounting Google Drive to Collab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
csv_folder_path = '/content/drive/My Drive/Data_Mining_CSV'

# **Part 1**

In [ ]:
!pip install datasets

Requirements

In [ ]:
from itertools import islice
import sys
import pandas as pd
import json
import re
from datasets import load_dataset

print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

In [ ]:
# ------------------- Cleaning Functions -------------------

def normalize_price(price):
    if isinstance(price, str):
        match = re.search(r'[\d,.]+', price)
        if match:
            return float(match.group(0).replace(',', ''))
    return None

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s]', '', text)
        return text.strip()
    return ''

def flatten_image_struct(example):
    images_raw = example.get('images', '[]')
    try:
        images = json.loads(images_raw) if isinstance(images_raw, str) else images_raw
    except json.JSONDecodeError:
        images = []
    example['image_urls'] = [img.get('hi_res', '') for img in images if isinstance(img, dict)]
    return example

# Home and Kitchen

In [ ]:
# ------------------- Load and Sample Data -------------------

Home_and_Kitchen_review_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Home_and_Kitchen", trust_remote_code=True, split="full", streaming=True)
Home_and_Kitchen_meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Home_and_Kitchen", trust_remote_code=True, split="full", streaming=True)

# Get set amount of rows
hak_rows_to_get = 20000
hak_review_sample = list(islice(Home_and_Kitchen_review_ds, hak_rows_to_get))
Home_and_Kitchen_meta_ds = Home_and_Kitchen_meta_ds.map(flatten_image_struct)
hak_meta_sample = list(islice(Home_and_Kitchen_meta_ds, hak_rows_to_get))
pd.set_option('display.max_rows', hak_rows_to_get)
pd.set_option('display.max_colwidth', None)

# # Get all rows
# hak_review_sample = list(Home_and_Kitchen_review_ds)
# Home_and_Kitchen_meta_ds = Home_and_Kitchen_meta_ds.map(flatten_image_struct)
# hak_meta_sample = list(Home_and_Kitchen_meta_ds)

hak_reviews_df = pd.DataFrame(hak_review_sample)
hak_meta_df = pd.DataFrame(hak_meta_sample)

# ------------------- Clean Meta Dataset -------------------

clean_hak_meta_df = hak_meta_df.copy()

meta_cols_to_keep = [
    'parent_asin', 'main_category', 'product_title', 'average_rating', 'rating_number',
    'description', 'price', 'store', 'details'
]
clean_hak_meta_df = clean_hak_meta_df.rename(columns={'title': 'product_title'})
clean_hak_meta_df = clean_hak_meta_df.loc[:, meta_cols_to_keep]

# Drop incomplete entries
clean_hak_meta_df = clean_hak_meta_df.dropna(subset=['product_title', 'main_category', 'price']).copy()

# Clean 'description'
clean_hak_meta_df.loc[:, 'description'] = clean_hak_meta_df['description'].apply(
    lambda desc: clean_text(' '.join(desc)) if isinstance(desc, list)
    else clean_text(desc) if isinstance(desc, str)
    else 'No description available'
)

# Clean other fields
clean_hak_meta_df.loc[:, 'details'] = clean_hak_meta_df['details'].fillna('').astype(str).apply(clean_text)
clean_hak_meta_df.loc[:, 'average_rating'] = clean_hak_meta_df['average_rating'].fillna(0).astype(float)
clean_hak_meta_df.loc[:, 'rating_number'] = clean_hak_meta_df['rating_number'].fillna(0).astype(int)
clean_hak_meta_df.loc[:, 'price'] = clean_hak_meta_df['price'].apply(normalize_price)
clean_hak_meta_df = clean_hak_meta_df.dropna(subset=['price']).copy()
clean_hak_meta_df.loc[:, 'store'] = clean_hak_meta_df['store'].fillna('Unknown')
clean_hak_meta_df.loc[:, 'product_title'] = clean_hak_meta_df['product_title'].apply(clean_text)
clean_hak_meta_df.loc[:, 'parent_asin'] = clean_hak_meta_df['parent_asin'].astype(str)

clean_hak_meta_df.reset_index(drop=True, inplace=True)

# ------------------- Clean Reviews Dataset -------------------

clean_hak_reviews_df = hak_reviews_df.copy()

if 'images' in clean_hak_reviews_df.columns:
    clean_hak_reviews_df = clean_hak_reviews_df.drop(columns=['images'])

clean_hak_reviews_df = clean_hak_reviews_df.dropna(subset=['rating', 'text', 'asin', 'parent_asin']).copy()
clean_hak_reviews_df = clean_hak_reviews_df[clean_hak_reviews_df['verified_purchase'] == True].copy()

clean_hak_reviews_df.loc[:, 'title'] = clean_hak_reviews_df['title'].fillna('').apply(clean_text)
clean_hak_reviews_df.loc[:, 'text'] = clean_hak_reviews_df['text'].apply(clean_text)
clean_hak_reviews_df.loc[:, 'helpful_vote'] = clean_hak_reviews_df['helpful_vote'].fillna(0).astype(int)

# Convert timestamp from milliseconds to seconds (remove milliseconds)
clean_hak_reviews_df.loc[:, 'timestamp'] = (clean_hak_reviews_df['timestamp'].astype('int64') // 1000)
converted_timestamps = pd.to_datetime(clean_hak_reviews_df['timestamp'], unit='s', errors='coerce')
clean_hak_reviews_df = clean_hak_reviews_df.drop(columns=['timestamp'])
clean_hak_reviews_df['timestamp'] = converted_timestamps

# Split timestamp into date and time
clean_hak_reviews_df.loc[:, 'date'] = clean_hak_reviews_df['timestamp'].dt.date
clean_hak_reviews_df.loc[:, 'time'] = clean_hak_reviews_df['timestamp'].dt.strftime('%H:%M:%S')

# Drop original timestamp column
clean_hak_reviews_df.drop(columns=['timestamp'], inplace=True)

clean_hak_reviews_df.loc[:, 'parent_asin'] = clean_hak_reviews_df['parent_asin'].astype(str)
clean_hak_reviews_df.rename(columns={'title': 'review_title'}, inplace=True)
clean_hak_reviews_df.reset_index(drop=True, inplace=True)

# ------------------- Merge Datasets -------------------

clean_hak_merged_df = pd.merge(clean_hak_reviews_df, clean_hak_meta_df, on='parent_asin', how='left')
clean_hak_merged_df = clean_hak_merged_df.dropna().reset_index(drop=True)

Save to CSV

In [ ]:
# Create the CSV file
clean_hak_merged_df.to_csv(f"{csv_folder_path}/Home_and_Kitchen.csv", index=False)

Load from CSV

In [ ]:
# Load Gift Cards CSV file
home_and_kitchen = pd.read_csv(f"{csv_folder_path}/Home_and_Kitchen.csv")

# Show file
home_and_kitchen.head()

# Automotive

In [ ]:
# ------------------- Load and Sample Data -------------------

Automotive_review_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Automotive", trust_remote_code=True, split="full", streaming=True)
Automotive_meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Automotive", trust_remote_code=True, split="full", streaming=True)

# Get set amount of rows
a_rows_to_get = 20000
a_review_sample = list(islice(Automotive_review_ds, a_rows_to_get))
Automotive_meta_ds = Automotive_meta_ds.map(flatten_image_struct)
a_meta_sample = list(islice(Automotive_meta_ds, a_rows_to_get))
pd.set_option('display.max_rows', a_rows_to_get)
pd.set_option('display.max_colwidth', None)

# # Get all rows
# a_review_sample = list(Automotive_review_ds)
# Automotive_meta_ds = Automotive_meta_ds.map(flatten_image_struct)
# a_meta_sample = list(Automotive_meta_ds)

a_reviews_df = pd.DataFrame(a_review_sample)
a_meta_df = pd.DataFrame(a_meta_sample)

# ------------------- Clean Meta Dataset -------------------

clean_a_meta_df = a_meta_df.copy()

meta_cols_to_keep = [
    'parent_asin', 'main_category', 'product_title', 'average_rating', 'rating_number',
    'description', 'price', 'store', 'details'
]
clean_a_meta_df = clean_a_meta_df.rename(columns={'title': 'product_title'})
clean_a_meta_df = clean_a_meta_df.loc[:, meta_cols_to_keep]

# Drop incomplete entries
clean_a_meta_df = clean_a_meta_df.dropna(subset=['product_title', 'main_category', 'price']).copy()

# Clean 'description'
clean_a_meta_df.loc[:, 'description'] = clean_a_meta_df['description'].apply(
    lambda desc: clean_text(' '.join(desc)) if isinstance(desc, list)
    else clean_text(desc) if isinstance(desc, str)
    else 'No description available'
)

# Clean other fields
clean_a_meta_df.loc[:, 'details'] = clean_a_meta_df['details'].fillna('').astype(str).apply(clean_text)
clean_a_meta_df.loc[:, 'average_rating'] = clean_a_meta_df['average_rating'].fillna(0).astype(float)
clean_a_meta_df.loc[:, 'rating_number'] = clean_a_meta_df['rating_number'].fillna(0).astype(int)
clean_a_meta_df.loc[:, 'price'] = clean_a_meta_df['price'].apply(normalize_price)
clean_a_meta_df = clean_a_meta_df.dropna(subset=['price']).copy()
clean_a_meta_df.loc[:, 'store'] = clean_a_meta_df['store'].fillna('Unknown')
clean_a_meta_df.loc[:, 'product_title'] = clean_a_meta_df['product_title'].apply(clean_text)
clean_a_meta_df.loc[:, 'parent_asin'] = clean_a_meta_df['parent_asin'].astype(str)

clean_a_meta_df.reset_index(drop=True, inplace=True)

# ------------------- Clean Reviews Dataset -------------------

clean_a_reviews_df = a_reviews_df.copy()

if 'images' in clean_a_reviews_df.columns:
    clean_a_reviews_df = clean_a_reviews_df.drop(columns=['images'])

clean_a_reviews_df = clean_a_reviews_df.dropna(subset=['rating', 'text', 'asin', 'parent_asin']).copy()
clean_a_reviews_df = clean_a_reviews_df[clean_a_reviews_df['verified_purchase'] == True].copy()

clean_a_reviews_df.loc[:, 'title'] = clean_a_reviews_df['title'].fillna('').apply(clean_text)
clean_a_reviews_df.loc[:, 'text'] = clean_a_reviews_df['text'].apply(clean_text)
clean_a_reviews_df.loc[:, 'helpful_vote'] = clean_a_reviews_df['helpful_vote'].fillna(0).astype(int)

# Convert timestamp from milliseconds to seconds (remove milliseconds)
clean_a_reviews_df.loc[:, 'timestamp'] = (clean_a_reviews_df['timestamp'].astype('int64') // 1000)
converted_timestamps = pd.to_datetime(clean_a_reviews_df['timestamp'], unit='s', errors='coerce')
clean_a_reviews_df = clean_a_reviews_df.drop(columns=['timestamp'])
clean_a_reviews_df['timestamp'] = converted_timestamps

# Split timestamp into date and time
clean_a_reviews_df.loc[:, 'date'] = clean_a_reviews_df['timestamp'].dt.date
clean_a_reviews_df.loc[:, 'time'] = clean_a_reviews_df['timestamp'].dt.strftime('%H:%M:%S')

# Drop original timestamp column
clean_a_reviews_df.drop(columns=['timestamp'], inplace=True)

clean_a_reviews_df.loc[:, 'parent_asin'] = clean_a_reviews_df['parent_asin'].astype(str)
clean_a_reviews_df.rename(columns={'title': 'review_title'}, inplace=True)
clean_a_reviews_df.reset_index(drop=True, inplace=True)

# ------------------- Merge Datasets -------------------

clean_a_merged_df = pd.merge(clean_a_reviews_df, clean_a_meta_df, on='parent_asin', how='left')
clean_a_merged_df = clean_a_merged_df.dropna().reset_index(drop=True)

Save to CSV

In [ ]:
# Create the CSV file
clean_a_merged_df.to_csv(f"{csv_folder_path}/Automotive.csv", index=False)

Load from CSV

In [ ]:
# Load Gift Cards CSV file
automotive_df = pd.read_csv(f"{csv_folder_path}/Automotive.csv")

# Show file
automotive_df.head()

# Electronics

In [ ]:
# ------------------- Load and Sample Data -------------------

Electronics_review_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Electronics", trust_remote_code=True, split="full", streaming=True)
Electronics_meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Electronics", trust_remote_code=True, split="full", streaming=True)

# Get set amount of rows
e_rows_to_get = 20000
e_review_sample = list(islice(Electronics_review_ds, e_rows_to_get))
Electronics_meta_ds = Electronics_meta_ds.map(flatten_image_struct)
e_meta_sample = list(islice(Electronics_meta_ds, e_rows_to_get))
pd.set_option('display.max_rows', e_rows_to_get)
pd.set_option('display.max_colwidth', None)

# # Get all rows
# e_review_sample = list(Electronics_review_ds)
# Electronics_meta_ds = Electronics_meta_ds.map(flatten_image_struct)
# e_meta_sample = list(Electronics_meta_ds)

e_reviews_df = pd.DataFrame(e_review_sample)
e_meta_df = pd.DataFrame(e_meta_sample)

# ------------------- Clean Meta Dataset -------------------

clean_e_meta_df = e_meta_df.copy()

meta_cols_to_keep = [
    'parent_asin', 'main_category', 'product_title', 'average_rating', 'rating_number',
    'description', 'price', 'store', 'details'
]
clean_e_meta_df = clean_e_meta_df.rename(columns={'title': 'product_title'})
clean_e_meta_df = clean_e_meta_df.loc[:, meta_cols_to_keep]

# Drop incomplete entries
clean_e_meta_df = clean_e_meta_df.dropna(subset=['product_title', 'main_category', 'price']).copy()

# Clean 'description'
clean_e_meta_df.loc[:, 'description'] = clean_e_meta_df['description'].apply(
    lambda desc: clean_text(' '.join(desc)) if isinstance(desc, list)
    else clean_text(desc) if isinstance(desc, str)
    else 'No description available'
)

# Clean other fields
clean_e_meta_df.loc[:, 'details'] = clean_e_meta_df['details'].fillna('').astype(str).apply(clean_text)
clean_e_meta_df.loc[:, 'average_rating'] = clean_e_meta_df['average_rating'].fillna(0).astype(float)
clean_e_meta_df.loc[:, 'rating_number'] = clean_e_meta_df['rating_number'].fillna(0).astype(int)
clean_e_meta_df.loc[:, 'price'] = clean_e_meta_df['price'].apply(normalize_price)
clean_e_meta_df = clean_e_meta_df.dropna(subset=['price']).copy()
clean_e_meta_df.loc[:, 'store'] = clean_e_meta_df['store'].fillna('Unknown')
clean_e_meta_df.loc[:, 'product_title'] = clean_e_meta_df['product_title'].apply(clean_text)
clean_e_meta_df.loc[:, 'parent_asin'] = clean_e_meta_df['parent_asin'].astype(str)

clean_e_meta_df.reset_index(drop=True, inplace=True)

# ------------------- Clean Reviews Dataset -------------------

clean_e_reviews_df = e_reviews_df.copy()

if 'images' in clean_e_reviews_df.columns:
    clean_e_reviews_df = clean_e_reviews_df.drop(columns=['images'])

clean_e_reviews_df = clean_e_reviews_df.dropna(subset=['rating', 'text', 'asin', 'parent_asin']).copy()
clean_e_reviews_df = clean_e_reviews_df[clean_e_reviews_df['verified_purchase'] == True].copy()

clean_e_reviews_df.loc[:, 'title'] = clean_e_reviews_df['title'].fillna('').apply(clean_text)
clean_e_reviews_df.loc[:, 'text'] = clean_e_reviews_df['text'].apply(clean_text)
clean_e_reviews_df.loc[:, 'helpful_vote'] = clean_e_reviews_df['helpful_vote'].fillna(0).astype(int)

# Convert timestamp from milliseconds to seconds (remove milliseconds)
clean_e_reviews_df.loc[:, 'timestamp'] = (clean_e_reviews_df['timestamp'].astype('int64') // 1000)
converted_timestamps = pd.to_datetime(clean_e_reviews_df['timestamp'], unit='s', errors='coerce')
clean_e_reviews_df = clean_e_reviews_df.drop(columns=['timestamp'])
clean_e_reviews_df['timestamp'] = converted_timestamps

# Split timestamp into date and time
clean_e_reviews_df.loc[:, 'date'] = clean_e_reviews_df['timestamp'].dt.date
clean_e_reviews_df.loc[:, 'time'] = clean_e_reviews_df['timestamp'].dt.strftime('%H:%M:%S')

# Drop original timestamp column
clean_e_reviews_df.drop(columns=['timestamp'], inplace=True)

clean_e_reviews_df.loc[:, 'parent_asin'] = clean_e_reviews_df['parent_asin'].astype(str)
clean_e_reviews_df.rename(columns={'title': 'review_title'}, inplace=True)
clean_e_reviews_df.reset_index(drop=True, inplace=True)

# ------------------- Merge Datasets -------------------

clean_e_merged_df = pd.merge(clean_e_reviews_df, clean_e_meta_df, on='parent_asin', how='left')
clean_e_merged_df = clean_e_merged_df.dropna().reset_index(drop=True)

Save to CSV

In [ ]:
# Create the CSV file
clean_e_merged_df.to_csv(f"{csv_folder_path}/Electronics.csv", index=False)

Load from CSV

In [ ]:
# Load Gift Cards CSV file
electronics_df = pd.read_csv(f"{csv_folder_path}/Electronics.csv")

# Show file
electronics_df.head()

# Health and Household

In [ ]:
# ------------------- Load and Sample Data -------------------

Health_and_Household_review_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Health_and_Household", trust_remote_code=True, split="full", streaming=True)
Health_and_Household_meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Health_and_Household", trust_remote_code=True, split="full", streaming=True)

# Get set amount of rows
hah_rows_to_get = 20000
hah_review_sample = list(islice(Health_and_Household_review_ds, hah_rows_to_get))
Health_and_Household_meta_ds = Health_and_Household_meta_ds.map(flatten_image_struct)
hah_meta_sample = list(islice(Health_and_Household_meta_ds, hah_rows_to_get))
pd.set_option('display.max_rows', hah_rows_to_get)
pd.set_option('display.max_colwidth', None)

# # Get all rows
# hah_review_sample = list(Health_and_Household_review_ds)
# Health_and_Household_meta_ds = Health_and_Household_meta_ds.map(flatten_image_struct)
# hah_meta_sample = list(Health_and_Household_meta_ds)

hah_reviews_df = pd.DataFrame(hah_review_sample)
hah_meta_df = pd.DataFrame(hah_meta_sample)

# ------------------- Clean Meta Dataset -------------------

clean_hah_meta_df = hah_meta_df.copy()

meta_cols_to_keep = [
    'parent_asin', 'main_category', 'product_title', 'average_rating', 'rating_number',
    'description', 'price', 'store', 'details'
]
clean_hah_meta_df = clean_hah_meta_df.rename(columns={'title': 'product_title'})
clean_hah_meta_df = clean_hah_meta_df.loc[:, meta_cols_to_keep]

# Drop incomplete entries
clean_hah_meta_df = clean_hah_meta_df.dropna(subset=['product_title', 'main_category', 'price']).copy()

# Clean 'description'
clean_hah_meta_df.loc[:, 'description'] = clean_hah_meta_df['description'].apply(
    lambda desc: clean_text(' '.join(desc)) if isinstance(desc, list)
    else clean_text(desc) if isinstance(desc, str)
    else 'No description available'
)

# Clean other fields
clean_hah_meta_df.loc[:, 'details'] = clean_hah_meta_df['details'].fillna('').astype(str).apply(clean_text)
clean_hah_meta_df.loc[:, 'average_rating'] = clean_hah_meta_df['average_rating'].fillna(0).astype(float)
clean_hah_meta_df.loc[:, 'rating_number'] = clean_hah_meta_df['rating_number'].fillna(0).astype(int)
clean_hah_meta_df.loc[:, 'price'] = clean_hah_meta_df['price'].apply(normalize_price)
clean_hah_meta_df = clean_hah_meta_df.dropna(subset=['price']).copy()
clean_hah_meta_df.loc[:, 'store'] = clean_hah_meta_df['store'].fillna('Unknown')
clean_hah_meta_df.loc[:, 'product_title'] = clean_hah_meta_df['product_title'].apply(clean_text)
clean_hah_meta_df.loc[:, 'parent_asin'] = clean_hah_meta_df['parent_asin'].astype(str)

clean_hah_meta_df.reset_index(drop=True, inplace=True)

# ------------------- Clean Reviews Dataset -------------------

clean_hah_reviews_df = hah_reviews_df.copy()

if 'images' in clean_hah_reviews_df.columns:
    clean_hah_reviews_df = clean_hah_reviews_df.drop(columns=['images'])

clean_hah_reviews_df = clean_hah_reviews_df.dropna(subset=['rating', 'text', 'asin', 'parent_asin']).copy()
clean_hah_reviews_df = clean_hah_reviews_df[clean_hah_reviews_df['verified_purchase'] == True].copy()

clean_hah_reviews_df.loc[:, 'title'] = clean_hah_reviews_df['title'].fillna('').apply(clean_text)
clean_hah_reviews_df.loc[:, 'text'] = clean_hah_reviews_df['text'].apply(clean_text)
clean_hah_reviews_df.loc[:, 'helpful_vote'] = clean_hah_reviews_df['helpful_vote'].fillna(0).astype(int)

# Convert timestamp from milliseconds to seconds (remove milliseconds)
clean_hah_reviews_df.loc[:, 'timestamp'] = (clean_hah_reviews_df['timestamp'].astype('int64') // 1000)
converted_timestamps = pd.to_datetime(clean_hah_reviews_df['timestamp'], unit='s', errors='coerce')
clean_hah_reviews_df = clean_hah_reviews_df.drop(columns=['timestamp'])
clean_hah_reviews_df['timestamp'] = converted_timestamps

# Split timestamp into date and time
clean_hah_reviews_df.loc[:, 'date'] = clean_hah_reviews_df['timestamp'].dt.date
clean_hah_reviews_df.loc[:, 'time'] = clean_hah_reviews_df['timestamp'].dt.strftime('%H:%M:%S')

# Drop original timestamp column
clean_hah_reviews_df.drop(columns=['timestamp'], inplace=True)

clean_hah_reviews_df.loc[:, 'parent_asin'] = clean_hah_reviews_df['parent_asin'].astype(str)
clean_hah_reviews_df.rename(columns={'title': 'review_title'}, inplace=True)
clean_hah_reviews_df.reset_index(drop=True, inplace=True)

# ------------------- Merge Datasets -------------------

clean_hah_merged_df = pd.merge(clean_hah_reviews_df, clean_hah_meta_df, on='parent_asin', how='left')
clean_hah_merged_df = clean_hah_merged_df.dropna().reset_index(drop=True)

Save to CSV

In [ ]:
# Create the CSV file
clean_hah_merged_df.to_csv(f"{csv_folder_path}/Health_and_Household.csv", index=False)

Load from CSV

In [ ]:
# Load Gift Cards CSV file
health_and_household_df = pd.read_csv(f"{csv_folder_path}/Health_and_Household.csv")

# Show file
health_and_household_df.head()

# Software

In [ ]:
# ------------------- Load and Sample Data -------------------

Software_review_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Software", trust_remote_code=True, split="full", streaming=True)
Software_meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Software", trust_remote_code=True, split="full", streaming=True)

# Get set amount of rows
s_rows_to_get = 20000
s_review_sample = list(islice(Software_review_ds, s_rows_to_get))
Software_meta_ds = Software_meta_ds.map(flatten_image_struct)
s_meta_sample = list(islice(Software_meta_ds, s_rows_to_get))
pd.set_option('display.max_rows', s_rows_to_get)
pd.set_option('display.max_colwidth', None)

# # Get all rows
# s_review_sample = list(Software_review_ds)
# Software_meta_ds = Software_meta_ds.map(flatten_image_struct)
# s_meta_sample = list(Software_meta_ds)

s_reviews_df = pd.DataFrame(s_review_sample)
s_meta_df = pd.DataFrame(s_meta_sample)

# ------------------- Clean Meta Dataset -------------------

clean_s_meta_df = s_meta_df.copy()

meta_cols_to_keep = [
    'parent_asin', 'main_category', 'product_title', 'average_rating', 'rating_number',
    'description', 'price', 'store', 'details'
]
clean_s_meta_df = clean_s_meta_df.rename(columns={'title': 'product_title'})
clean_s_meta_df = clean_s_meta_df.loc[:, meta_cols_to_keep]

# Drop incomplete entries
clean_s_meta_df = clean_s_meta_df.dropna(subset=['product_title', 'main_category', 'price']).copy()

# Clean 'description'
clean_s_meta_df.loc[:, 'description'] = clean_s_meta_df['description'].apply(
    lambda desc: clean_text(' '.join(desc)) if isinstance(desc, list)
    else clean_text(desc) if isinstance(desc, str)
    else 'No description available'
)

# Clean other fields
clean_s_meta_df.loc[:, 'details'] = clean_s_meta_df['details'].fillna('').astype(str).apply(clean_text)
clean_s_meta_df.loc[:, 'average_rating'] = clean_s_meta_df['average_rating'].fillna(0).astype(float)
clean_s_meta_df.loc[:, 'rating_number'] = clean_s_meta_df['rating_number'].fillna(0).astype(int)
clean_s_meta_df.loc[:, 'price'] = clean_s_meta_df['price'].apply(normalize_price)
clean_s_meta_df = clean_s_meta_df.dropna(subset=['price']).copy()
clean_s_meta_df.loc[:, 'store'] = clean_s_meta_df['store'].fillna('Unknown')
clean_s_meta_df.loc[:, 'product_title'] = clean_s_meta_df['product_title'].apply(clean_text)
clean_s_meta_df.loc[:, 'parent_asin'] = clean_s_meta_df['parent_asin'].astype(str)

clean_s_meta_df.reset_index(drop=True, inplace=True)

# ------------------- Clean Reviews Dataset -------------------

clean_s_reviews_df = s_reviews_df.copy()

if 'images' in clean_s_reviews_df.columns:
    clean_s_reviews_df = clean_s_reviews_df.drop(columns=['images'])

clean_s_reviews_df = clean_s_reviews_df.dropna(subset=['rating', 'text', 'asin', 'parent_asin']).copy()
clean_s_reviews_df = clean_s_reviews_df[clean_s_reviews_df['verified_purchase'] == True].copy()

clean_s_reviews_df.loc[:, 'title'] = clean_s_reviews_df['title'].fillna('').apply(clean_text)
clean_s_reviews_df.loc[:, 'text'] = clean_s_reviews_df['text'].apply(clean_text)
clean_s_reviews_df.loc[:, 'helpful_vote'] = clean_s_reviews_df['helpful_vote'].fillna(0).astype(int)

# Convert timestamp from milliseconds to seconds (remove milliseconds)
clean_s_reviews_df.loc[:, 'timestamp'] = (clean_s_reviews_df['timestamp'].astype('int64') // 1000)
converted_timestamps = pd.to_datetime(clean_s_reviews_df['timestamp'], unit='s', errors='coerce')
clean_s_reviews_df = clean_s_reviews_df.drop(columns=['timestamp'])
clean_s_reviews_df['timestamp'] = converted_timestamps

# Split timestamp into date and time
clean_s_reviews_df.loc[:, 'date'] = clean_s_reviews_df['timestamp'].dt.date
clean_s_reviews_df.loc[:, 'time'] = clean_s_reviews_df['timestamp'].dt.strftime('%H:%M:%S')

# Drop original timestamp column
clean_s_reviews_df.drop(columns=['timestamp'], inplace=True)

clean_s_reviews_df.loc[:, 'parent_asin'] = clean_s_reviews_df['parent_asin'].astype(str)
clean_s_reviews_df.rename(columns={'title': 'review_title'}, inplace=True)
clean_s_reviews_df.reset_index(drop=True, inplace=True)

# ------------------- Merge Datasets -------------------

clean_s_merged_df = pd.merge(clean_s_reviews_df, clean_s_meta_df, on='parent_asin', how='left')
clean_s_merged_df = clean_s_merged_df.dropna().reset_index(drop=True)

Save to CSV

In [ ]:
# Create the CSV file
clean_s_merged_df.to_csv(f"{csv_folder_path}/Software.csv", index=False)

Load from CSV

In [ ]:
# Load Gift Cards CSV file
software_df = pd.read_csv(f"{csv_folder_path}/Software.csv")

# Show file
software_df.head()